In [1]:
# Import the required modules
# %pylab inline 
import cv2
import numpy as np
import IPython
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from pylab import rcParams
import threading
from datetime import datetime
import pytz
import glob

# initialization
PATH_TO_TRAIN_IMAGES_DIR = '/Users/hemingwei/Documents/MUM/ML/research/Gesture/data/gesture/training_set_variance'
TRAIN_IMAGE_PATHS = glob.glob(PATH_TO_TRAIN_IMAGES_DIR + '/*.png')

frame = None
nums = [0,0,0,0,0,0]
timerNum = 0
flashNum = 0
curCls = 0
total_flash_foreach = 3
seconds_for_preparation = 3
run = False

imgs_cache = []
imgs_mini = []

def updateCounter(numstr):
    nums[numstr] += 1

def save():
    global imgs_mini
    global imgs_cache
    global run
    for i in range(6):
        filename = PATH_TO_TRAIN_IMAGES_DIR + "/" + str(i) + "-" \
         + datetime.now(pytz.timezone("America/Chicago")).strftime('%Y%m%d%H%M%S%f')[:-3] + ".png"
        cv2.imwrite(filename, imgs_cache[i])
        updateCounter(i)
    imgs_mini = []
    imgs_cache = []
    run = False
        
def collectData():
    global imgs_mini
    global imgs_cache
    _f = frame[90:390, 135:505]
    if len(imgs_mini) < 6:
        imgs_mini.append(cv2.resize(_f, (0,0), fx=0.2, fy=0.2))
        imgs_cache.append(_f)
    
def readyToShot():
    global flashNum
    global curCls
    global run
    if run:
        if curCls < 6:
            if flashNum != 0:
                flashNum -= 1
            else:
                collectData()
                curCls += 1
                if curCls < 6:
                    flashNum = total_flash_foreach * 2
            threading.Timer(0.5, readyToShot).start()
        else:
            curCls = 0
        
def prepareToShot():
    global timerNum
    global curCls
    global flashNum
    global run
    if run:
        if timerNum != 0:
            timerNum -= 1
            threading.Timer(1, prepareToShot).start()
        else:
            flashNum = total_flash_foreach * 2
            readyToShot()

# initialize statistic data
for image_path in TRAIN_IMAGE_PATHS:
    cls_ = image_path[image_path.rfind('/')+1:image_path.index('-')]
    updateCounter(int(cls_))
        
stype = cv2.FONT_HERSHEY_SIMPLEX
# red:(0,0,255) blue:(255,0,0) pink:(255,0,255) black:(0,0,0) white:(255,255,255) green: (0,255,0)
title_color = (255,0,255)
content_color = (0,0,255)
sclale = .5
thickness = 1

cap = cv2.VideoCapture(0)
cap.set(3, 640);
cap.set(4, 480);
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    size = len(imgs_mini)
    if size != 0:
        frame[0:size*60, 566:640] = np.concatenate(imgs_mini, 0)
    if size == 6:
        cv2.rectangle(frame,(222,135),(312,180),(0,255,0),3)
        cv2.rectangle(frame,(322,135),(418,180),(0,255,0),3)
        cv2.putText(frame,'Save[Y]', (235,160), stype, .5, title_color, 2)
        cv2.putText(frame,'Cancel[N]', (332,160), stype, .5, title_color, 2)
    # Display the resulting frame
    cv2.rectangle(frame,(132,85),(508,392),(0,255,0),3)
    cv2.putText(frame,'Data collection statictic:', (10,15), stype, .5, title_color, thickness)
    cv2.putText(frame,'G0: ', (10,30), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[0]), (40,30), stype, sclale, content_color, thickness)
    cv2.putText(frame,'G1: ', (10,45), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[1]), (40,45), stype, sclale, content_color, thickness)
    cv2.putText(frame,'G2: ', (10,60), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[2]), (40,60), stype, sclale, content_color, thickness)
    cv2.putText(frame,'G3: ', (10,75), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[3]), (40,75), stype, sclale, content_color, thickness)
    cv2.putText(frame,'G4: ', (10,90), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[4]), (40,90), stype, sclale, content_color, thickness)
    cv2.putText(frame,'G5: ', (10,105), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[5]), (40,105), stype, sclale, content_color, thickness)
    if timerNum:
        cv2.putText(frame, str(timerNum), (270,285), stype, 5, (255,255,255), 7)
    if flashNum % 2 == 1:
        cv2.putText(frame, str(curCls), (270,285), stype, 5, (255,255,255), 7)
    # show
    cv2.imshow('frame',frame)
    k = chr(cv2.waitKey(1) & 0xFF)
    if k == 'q':
        run = False
        break
    elif k == ' ':
        if not run:
            run = True
            imgs_mini = []
            imgs_cache = []
            timerNum = seconds_for_preparation
            threading.Timer(1, prepareToShot).start()
    elif k == 'y' and size == 6:
        threading.Thread(target=save).start()
    elif k == 'n' and size == 6:
        imgs_mini = []
        imgs_cache = []
        run = False
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

{'restart': True, 'status': 'ok'}